# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [2]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

from google.colab import files
uploaded = files.upload()

#data_path = 'data/'
#df = pd.read_csv(data_path + 'titanic_train.csv')
df = pd.read_csv('titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

Saving titanic_train.csv to titanic_train.csv


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
df['Cabin'] = df['Cabin'].fillna('None')
mean_df = df.groupby(['Cabin'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Cabin'])['Fare'].apply(lambda x: x.mode()[0]).reset_index() #中位數沒有函數可用
median_df = df.groupby(['Cabin'])['Fare'].median().reset_index()
max_df = df.groupby(['Cabin'])['Fare'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Cabin'])
temp = pd.merge(temp, median_df, how='left', on=['Cabin'])
temp = pd.merge(temp, max_df, how='left', on=['Cabin'])
temp.columns = ['Cabin', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max']
temp.head(10)

,Cabin,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,A10,40.1250,40.1250,40.1250,40.1250
1,A14,52.0000,52.0000,52.0000,52.0000
2,A16,39.6000,39.6000,39.6000,39.6000
3,A19,26.0000,26.0000,26.0000,26.0000
4,A20,56.9292,56.9292,56.9292,56.9292
5,A23,30.0000,30.0000,30.0000,30.0000
6,A24,50.4958,50.4958,50.4958,50.4958
7,A26,35.5000,35.5000,35.5000,35.5000
8,A31,31.0000,31.0000,31.0000,31.0000
9,A32,50.0000,50.0000,50.0000,50.0000


In [7]:
df = pd.merge(df, temp, how='left', on=['Cabin'])
df = df.drop(['Cabin'] , axis=1) #為什麼Cabin刪掉?
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,19.157325,8.0500,10.5000,512.3292
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,71.283300,71.2833,71.2833,71.2833
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,19.157325,8.0500,10.5000,512.3292
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,53.100000,53.1000,53.1000,53.1000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,19.157325,8.0500,10.5000,512.3292


In [8]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max']



,Pclass,Age,SibSp,Parch,Fare,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,3,22.0,1,0,7.2500,19.157325,8.0500,10.5000,512.3292
1,1,38.0,1,0,71.2833,71.283300,71.2833,71.2833,71.2833
2,3,26.0,0,0,7.9250,19.157325,8.0500,10.5000,512.3292
3,1,35.0,1,0,53.1000,53.100000,53.1000,53.1000,53.1000
4,3,35.0,0,0,8.0500,19.157325,8.0500,10.5000,512.3292


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?
ANS. 由0.6892 -> 0.7038

In [9]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
# 沒有這四個新特徵的 dataframe 稱為 df_minus
df_minus = df.drop(['Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max'] , axis=1)

# 原始特徵 + 線性迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warni

0.7038635542329971

In [10]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warni

0.6892504646627564